In [11]:
from bs4 import BeautifulSoup
from collections import defaultdict
import requests

dom = BeautifulSoup(request_html('http://www.ceca.uwaterloo.ca/students/sessions.php?month_num=1&year_num=2017'), 'html.parser')
len(dom.select('td p a'))

companies = defaultdict(dict)
for year in range(2006, 2018):
    for month in range(1, 13):
        url = 'http://www.ceca.uwaterloo.ca/students/sessions.php?month_num={month}&year_num={year}'.format(month=month, year=year)
        dom = BeautifulSoup(request_html(url), 'html.parser')
        calendar = []
        for i in dom.select('td p a'):
            calendar.append(i.text)
        companies[year][month] = calendar
print(companies)

defaultdict(<type 'dict'>, {2016: {1: [u'No info sessions', u"New Year's Day", u'No info sessions', u'Lectures begin', u'Mattermost Inc.', u'Google', u'Deloitte', u'Twitter', u'Venture For Canada', u'Google', u'Capital One', u'500px', u'CIBC Wholesale Banking', u'Siemens Canada', u'Protiviti (CPA and non-CPA roles)', u'Wish', u'*CANCELLED PagerDuty Inc.', u'TunnelBear Inc.', u'Airbnb', u'*CANCELLED Yext', u'Inertia Engineering + Design', u'Genesys Laboratories', u'Kik Interactive', u'Sandvine', u'Aviva Canada', u'Ubisoft Toronto', u'Palantir', u'Sun Life Financial - Technology Opportunities', u'Bloomberg LP', u'Infusion', u'Influitive ', u'TextNow', u'Suncor Energy ', u'Microsoft', u'Meraki', u'Digiflare Inc.', u'MAIN POSTING 1 OPENS', u'Fairfax Financial Holdings Ltd.', u'*CANCELLED Uken Games', u'Amazon', u'Desjardins General Insurance Group', u'McRae Integration Ltd.', u'Noom, Inc.', u'MAIN POSTING 1 CLOSES', u'D2L', u'**CANCELLED - TATA Consutancy Services', u'Loblaw Digital', u'A 

In [5]:
def request_html(url):
    """GET request to blog link, returns decoded HTML."""
    try:
        r = requests.get(url)
        return r.content
    except Exception as e:
        print(e)
        print('Error: could not GET from ' + url)
        return None

In [13]:
for i in companies:
    for j in companies[i]:
        print(companies[i][j])

[u'No info sessions', u"New Year's Day", u'No info sessions', u'Lectures begin', u'Mattermost Inc.', u'Google', u'Deloitte', u'Twitter', u'Venture For Canada', u'Google', u'Capital One', u'500px', u'CIBC Wholesale Banking', u'Siemens Canada', u'Protiviti (CPA and non-CPA roles)', u'Wish', u'*CANCELLED PagerDuty Inc.', u'TunnelBear Inc.', u'Airbnb', u'*CANCELLED Yext', u'Inertia Engineering + Design', u'Genesys Laboratories', u'Kik Interactive', u'Sandvine', u'Aviva Canada', u'Ubisoft Toronto', u'Palantir', u'Sun Life Financial - Technology Opportunities', u'Bloomberg LP', u'Infusion', u'Influitive ', u'TextNow', u'Suncor Energy ', u'Microsoft', u'Meraki', u'Digiflare Inc.', u'MAIN POSTING 1 OPENS', u'Fairfax Financial Holdings Ltd.', u'*CANCELLED Uken Games', u'Amazon', u'Desjardins General Insurance Group', u'McRae Integration Ltd.', u'Noom, Inc.', u'MAIN POSTING 1 CLOSES', u'D2L', u'**CANCELLED - TATA Consutancy Services', u'Loblaw Digital', u'A Thinking Ape Entertainment', u'*CANCEL

In [93]:
class Event:
    def __init__(self, title, year, month):
        self._title = title
        self.year = year
        self.month = month
        
    def is_valid(self):
        if 'No info sessions' in self._title:
            return False
        if 'MAIN ' in self._title:
            return False
        if 'Reading week' in self._title:
            return False
        if 'Info Session' in self._title:
            return False
        return True
       
    @property
    def title(self):
        return self._title.lower().replace('cancelled', '').replace('*', '').replace('-', '').strip()

In [94]:
e = Event('Cancelled', 1, 1)
e.title

''

In [95]:
terms = []
for i in companies:
    for j in range(0, 3):
        terms.append(companies[i][j*4+1] + companies[i][j*4+4] + companies[i][j*4+2] + companies[i][j*4+3])
terms

[[u'No info sessions',
  u"New Year's Day",
  u'No info sessions',
  u'Lectures begin',
  u'Mattermost Inc.',
  u'Google',
  u'Deloitte',
  u'Twitter',
  u'Venture For Canada',
  u'Google',
  u'Capital One',
  u'500px',
  u'CIBC Wholesale Banking',
  u'Siemens Canada',
  u'Protiviti (CPA and non-CPA roles)',
  u'Wish',
  u'*CANCELLED PagerDuty Inc.',
  u'TunnelBear Inc.',
  u'Airbnb',
  u'*CANCELLED Yext',
  u'Inertia Engineering + Design',
  u'Genesys Laboratories',
  u'Kik Interactive',
  u'Sandvine',
  u'Aviva Canada',
  u'Ubisoft Toronto',
  u'Palantir',
  u'Sun Life Financial - Technology Opportunities',
  u'Bloomberg LP',
  u'Infusion',
  u'Influitive ',
  u'TextNow',
  u'Suncor Energy ',
  u'Microsoft',
  u'Meraki',
  u'Digiflare Inc.',
  u'MAIN POSTING 1 OPENS',
  u'Fairfax Financial Holdings Ltd.',
  u'*CANCELLED Uken Games',
  u'Amazon',
  u'Desjardins General Insurance Group',
  u'McRae Integration Ltd.',
  u'Noom, Inc.',
  u'MAIN POSTING 1 CLOSES',
  u'D2L',
  u'**CANCELLED

In [96]:
events = []
for i in companies:
    for j in companies[i]:
        for item in companies[i][j]:
            events.append(Event(item, i, j))
events = filter(lambda x: x.is_valid(), events)
print(len(events))

3334


In [115]:
def group_by_term(events):
    groups = defaultdict(dict)
    for i in range(2006, 2018):
        for j in range(0, 3):
            groups[i][j] = []
            
    for e in events:
        groups[e.year][(e.month-1) / 4].append(e)
    flat_terms = []
    for i in range(2006, 2018):
        for j in range(0, 3):
            if j == 0:
                flat_terms.append(groups[i][j])
    return flat_terms
            

In [116]:
terms = group_by_term(events)

In [117]:
for i in terms:
    print(len(i))

94
112
126
85
117
140
156
151
159
183
163
168


In [118]:
all_companies = []
for i in terms:
    for e in i:
        all_companies.append(e.title)
all_companies = set(all_companies)

In [119]:
count = defaultdict(int)
for t in terms:
    for e in t:
        count[e.title] += 1
#for w in sorted(count, key=count.get, reverse=True):
#    print w, count[w]

In [120]:
tech_companies = ['microsoft', 'facebook', 'amazon', 'google', 'amazon', 'yelp', 'linkedin', 'twiter', 'palantir', 'hulu', 'yext', 'wish', 'nvidia', 'meraki', 'yahoo', 'shopify', 'pinterest', 'vmware', 'dropbox', 'mozilla', 'noom', 'stripe', 'atomic', 'zenefits', 'two sigma', 'tesla', 'pagerduty', 'snap', 'expedia', 'jane street', 'wealthsimple', 'quora', 'square']

In [121]:
company_matches = []
for t in terms:
    matches = []
    for e in t:
        for tc in tech_companies:
            if tc in e.title:
                matches.append(tc)
    company_matches.append(set(matches))

In [122]:
company_matches

[{'amazon', 'expedia', 'google', 'microsoft', 'nvidia', 'vmware'},
 {'amazon', 'expedia', 'google', 'microsoft', 'vmware', 'yahoo'},
 {'amazon',
  'expedia',
  'facebook',
  'google',
  'hulu',
  'microsoft',
  'nvidia',
  'vmware',
  'yahoo'},
 {'amazon',
  'expedia',
  'facebook',
  'google',
  'hulu',
  'microsoft',
  'mozilla',
  'yahoo'},
 {'amazon', 'facebook', 'google', 'hulu', 'microsoft', 'mozilla', 'yelp'},
 {'amazon',
  'facebook',
  'google',
  'hulu',
  'linkedin',
  'microsoft',
  'mozilla',
  'nvidia',
  'palantir',
  'shopify',
  'square',
  'yelp',
  'yext'},
 {'amazon',
  'facebook',
  'google',
  'hulu',
  'linkedin',
  'microsoft',
  'mozilla',
  'noom',
  'palantir',
  'square',
  'vmware',
  'yelp',
  'yext'},
 {'amazon',
  'facebook',
  'google',
  'hulu',
  'linkedin',
  'microsoft',
  'noom',
  'palantir',
  'pinterest',
  'snap',
  'tesla',
  'vmware',
  'yelp',
  'yext'},
 {'amazon',
  'dropbox',
  'facebook',
  'google',
  'hulu',
  'linkedin',
  'meraki',
 